In [7]:
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/translated/bleu_table.csv', index_col=0)

## Filter problematic rows

In [3]:
filtered = data[data.len > 10]
filtered = filtered.drop(columns=['T2-T3_en', 'T2-T3_es', 'T2-T3_fr', 'T2-T3_de'])

In [4]:
filtered

,T0-T1_en,T1-T2_en,T0-T1_es,T1-T2_es,T0-T1_fr,T1-T2_fr,T0-T1_de,T1-T2_de,src,origin,len
1,1.000000e+00,1.000000,0.495827,1.000000,0.0,0.0,0.631692,1.000000,2.0,0.0,17.0
3,7.361703e-01,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,2.0,0.0,15.0
5,7.309044e-01,0.902514,0.850931,1.000000,0.0,0.0,0.874341,1.000000,2.0,0.0,30.0
6,8.114530e-01,1.000000,1.000000,1.000000,0.0,0.0,0.550258,0.921960,2.0,0.0,46.0
8,2.586714e-11,0.658037,0.819978,0.857573,0.0,0.0,0.611736,0.440588,2.0,0.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...
59995,1.000000e+00,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,2.0,3.0,25.0
59996,6.734648e-01,0.750769,0.805558,1.000000,0.0,0.0,0.648288,0.634192,2.0,3.0,24.0
59997,7.186970e-01,0.625654,0.564882,1.000000,0.0,0.0,0.401919,0.509459,2.0,3.0,26.0
59998,5.060645e-01,1.000000,1.000000,1.000000,0.0,0.0,0.769192,1.000000,2.0,3.0,31.0


In [16]:
norm_data = pd.get_dummies(filtered, columns=['src', 'origin'])
scaler = MinMaxScaler()
lengths = scaler.fit_transform(norm_data['len'].values.reshape(-1, 1))
norm_data['len'] = lengths
norm_data

,T0-T1_en,T1-T2_en,T0-T1_es,T1-T2_es,T0-T1_fr,T1-T2_fr,T0-T1_de,T1-T2_de,len,src_0.0,src_1.0,src_2.0,src_3.0,origin_0.0,origin_1.0,origin_2.0,origin_3.0
1,1.000000e+00,1.000000,0.495827,1.000000,0.0,0.0,0.631692,1.000000,0.027907,0,0,1,0,1,0,0,0
3,7.361703e-01,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,0.018605,0,0,1,0,1,0,0,0
5,7.309044e-01,0.902514,0.850931,1.000000,0.0,0.0,0.874341,1.000000,0.088372,0,0,1,0,1,0,0,0
6,8.114530e-01,1.000000,1.000000,1.000000,0.0,0.0,0.550258,0.921960,0.162791,0,0,1,0,1,0,0,0
8,2.586714e-11,0.658037,0.819978,0.857573,0.0,0.0,0.611736,0.440588,0.367442,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1.000000e+00,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,0.065116,0,0,1,0,0,0,0,1
59996,6.734648e-01,0.750769,0.805558,1.000000,0.0,0.0,0.648288,0.634192,0.060465,0,0,1,0,0,0,0,1
59997,7.186970e-01,0.625654,0.564882,1.000000,0.0,0.0,0.401919,0.509459,0.069767,0,0,1,0,0,0,0,1
59998,5.060645e-01,1.000000,1.000000,1.000000,0.0,0.0,0.769192,1.000000,0.093023,0,0,1,0,0,0,0,1


## Prepare and split data

In [5]:
y = filtered.origin
X = filtered.loc[:, filtered.columns != 'origin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
classifier = svm.SVC(probability=True)
classifier.fit(X_train, y_train)

SVC(probability=True)

In [8]:
classifier.score(X_test, y_test)

0.33130841121495325